In [2]:
import numpy as np
import networkx as nx
import time
import math
import pandas as pd
import scipy as sp
import plotly.express as px
import plotly.graph_objs as go
from scipy.sparse import *
from scipy import linalg
from scipy.sparse.linalg import norm
from scipy.optimize import least_squares

## Arnoldi 

This is a copy of the algorithm defined and tested in the notebook `algo2_testing`. It's an implementation of the Algorithm 2 from the paper. It's needed in this notebook since this function is called by the `algo4` function. It's implemented to return exactly what's needed in the `algo4` function.

Everything will be reorganized in the main.py file once everything is working.

In [28]:
def Arnoldi(A,v0,m):
    v = v0
    beta = np.linalg.norm(v)
    v = v/beta
    H = sp.sparse.lil_matrix((m+1,m)) 
    V = sp.sparse.lil_matrix((A.shape[0],m+1))
    V[:,0] = v # each column of V is a vector v

    for j in range(m):
        w = A @ v  
        for i in range(j):
            H[i,j] = v.T @ w # tmp is a 1x1 matrix, so it's O(1) in memory
            w = w - H[i,j]*v 
            
        H[j+1,j] = np.linalg.norm(w)

        if H[j+1,j] == 0:
            # print("Arnoldi breakdown")
            m = j
            v = 0
            break
        else:
            if j < m-1:
                v = w/H[j+1,j]
                V[:,j+1] = v

    return V, H, beta, j  

# Algorithm 4 testing

This algorithm is based on the "Algorithm 4" of the paper, the pseudocode provided by the authors is the following 

![](https://i.imgur.com/H92fru7.png)

In [1]:
def compute_gamma(res, a, k): # function to compute gamma
    gamma = np.ones(len(a))
    for i in range(len(a)):
        if i != k:
            gamma[i] = (res[i]*a[k])/(res[k]*a[i])
        else:
            gamma[i] = 0
    return gamma

Basic test case with random numbers to test the algorithm.

In [35]:
n = 1000
m = 1100
tau = 1e-6
a = [0.85, 0.88, 0.9, 0.95]

x = sp.sparse.lil_matrix((n,1))
x[0,0] = 1

# generate a random graph
G = nx.gnp_random_graph(n, 0.1)
v = np.repeat(1.0 / 1000, 1000) # p is the personalization vector
v = v.reshape(v.shape[0],1)

A = nx.to_scipy_sparse_array(G, dtype=float)
S = A.sum(axis=1) # S[i] is the sum of the weights of edges going out of node i
S[S != 0] = 1.0 / S[S != 0] # S[i] is now the sum of the weights of edges going into node i
Q = sp.sparse.csr_array(sp.sparse.spdiags(S.T, 0, *A.shape)) # Q is the matrix of edge weights

In [83]:
def Algo4(Pt, v, m, a: list, tau, maxit: int, x):

    mv, iter = 0, 1 # mv is the number of matrix-vector products, iter is the number of iterations
    
    # initialize x as a random sparse matrix. Each col is the pagerank vector for a different alpha
    x = sp.sparse.lil_matrix((Pt.shape[0], len(a)))


    # compute the residual vector, it is a matrix of size (n, len(a)). Each col is the residual vector for a different alpha. 
    I = sp.sparse.eye(Pt.shape[0], Pt.shape[1], format='lil')
    r = sp.sparse.lil_matrix((Pt.shape[0], len(a)))
    res = np.zeros(len(a))

    for i in range(len(a)):
        r[:,[i]] = sp.sparse.linalg.spsolve(I - a[i]*Pt, v)
        col = r[:,[i]].toarray()
        res[i] = np.linalg.norm(col)

    for _ in range(maxit):
        # check if we have converged
        err = np.absolute(np.amax(res))
        if err < tau:
            print("Computation ended successfully in ", iter, " iterations and ", mv, " matrix-vector products.")
            return x, iter, mv

        print("\niter = ", iter)
        print("res: ", res)
        print("err = ", err)


        k = int(np.argmax(res))
        print("k = ", k)
        gamma = compute_gamma(res, a, k)
       
        # Run Arnoldi
        r_k = r[:,[k]].toarray()
        A_arnoldi = (1/a[k])*I - Pt
        V, H, beta, j = Arnoldi((1/a[k])*I - Pt, r_k, m)
        H = H[:-1,:]
        V = V[:,:-1]
        mv = mv + j

        H_e1 = np.zeros(H.shape[0])
        H_e1[0] = 1

        # compute y as the minimizer of || beta*e1 - Hy ||_2 using the least squares method
        y = sp.sparse.lil_matrix((H.shape[1],len(a)))
        y[:,[k]] = sp.sparse.linalg.lsqr(H, beta*H_e1)[0]
        y_k = y[:,[k]].toarray()

        # # Update x
        x_new = x
        x_new[:,[k]] = x[:,[k]] + V @ y_k

        # Update res[k]
        V_e1 = np.zeros(V.shape[0])
        V_e1[0] = 1

        norm_k =np.linalg.norm(beta*V_e1 - V @ y_k)
        res[k] = a[k]*norm_k

        # multi shift
        for i in range(len(a)):
            if res[i] >= tau:
                # print("res[", i, "] is larger than tau = ", tau)

                # # Compute H as described in the paper
                # H_k = H[:,[k]].toarray()
                # H_i = H_k + ((1-a[i])/a[i] - (1-a[k])/a[k])
                # H[:,[i]] = H_i
                H = H + ((1-a[i])/a[i] - (1-a[k])/a[k])*sp.sparse.eye(H.shape[0], H.shape[1], format='lil')

                # Compute z as described in the paper
                z1 = H_e1*beta
                z1 = z1.reshape(z1.shape[0],1)
                z2 = H @ y[:,[1]]
                z2 = z2.reshape(z2.shape[0],1)
                z = z1 - z2

                # Solve the linear system 
                A = sp.sparse.hstack([H, z])
                b = (beta*H_e1)
                b = b.reshape(b.shape[0],1)
                # use the least squares method to solve the linear system
                to_split = sp.sparse.linalg.lsqr(A, b)[0]
                
                # the last element of y_to_split is the last element of gamma[i], the other elements are the elements of y[:[i]]
                y[:,[i]] = to_split[:-1]
                gamma[i] = to_split[-1]

                # update x
                x_new[:,i] = x[:,i] + V @ y[:,[i]]

                # update the residual vector
                res[i] = (a[i]/a[k])*pow(gamma[i], i)*res[k]

            else:
                if res[i] < tau:
                    print("res[", i, "] is smaller than tau = ", tau, " at iteration ", iter)

        iter = iter + 1
        x = x_new

    raise Exception('Maximum number of iterations reached')

In [84]:
x, iter, mv = Algo4(Q, v, m, a, tau, 100, x)


iter =  1
res:  [0.03189738 0.03190716 0.03191369 0.03193001]
err =  0.031930006625941795
k =  0

iter =  2
res:  [1.11728737e+00 8.26005227e-04 5.55288870e-10 4.81520495e-13]
err =  1.1172873666904701
k =  3
res[ 2 ] is smaller than tau =  1e-06  at iteration  2

iter =  3
res:  [1.17714008e+00 1.29941354e-03 5.55288870e-10 1.93969263e-18]
err =  1.1771400826095457
k =  3
res[ 2 ] is smaller than tau =  1e-06  at iteration  3

iter =  4
res:  [1.17714008e+00 1.29941354e-03 5.55288870e-10 1.93969263e-18]
err =  1.1771400826095457
k =  3
res[ 2 ] is smaller than tau =  1e-06  at iteration  4

iter =  5
res:  [1.17714008e+00 1.29941354e-03 5.55288870e-10 1.93969263e-18]
err =  1.1771400826095457
k =  3
res[ 2 ] is smaller than tau =  1e-06  at iteration  5

iter =  6
res:  [1.17714008e+00 1.29941354e-03 5.55288870e-10 1.93969263e-18]
err =  1.1771400826095457
k =  3


KeyboardInterrupt: 